# 5분봉 3틱룰

## 상승이 온뒤 조정이 왔을 때 바닥(반등전)을 찾는 방법으로 5분봉 15분봉을 주로 사용
1. 양봉 -> 음봉으로 전활할때는 카운팅 x
2. 그 다음봉도 음봉이고 이전봉대비 일정값 이상 차이가 나게되면 1틱으로 인정
3. 이전봉 마감가보다 높거나 차이가 없으면 무시한다. (그날그날 떨어지는 강도를 보고 1틱을 판단) 
4. 법칙 1~3에 의해 3틱이 채워지면, 최초 매소 진입!
5. 5분이 다 지나서 다음봉이 갱신되기 직전에 차트 모양을 보고 진입 여부를 판단
 + 음봉 사이에 양봉이 나왔을 경우 양봉이 추세를 무너트릴 정도로 강하면 초기화 But 양봉이 이전 양봉과 큰차이가 없다면 같이 묶어서 1틱으로 본다. 

In [ ]:
pip install --upgrade mplfinance

In [485]:
import pyupbit
import pyfolio as pf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import time
import pandas as pd
from sklearn.model_selection import train_test_split
import mplfinance as mpf

mpl.rc('font', family='NanumGothic')
mpl.rc('axes', unicode_minus = False)
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 100)

In [523]:
# 데이터를 불러온다.
def getdata(ticker="KRW-BTC", count = 200, date=None, interval = "minute5"):
    dfs = [ ]
    for i in range(count // 200 + 1):
        if i < count // 200 :
            df = pyupbit.get_ohlcv(ticker, to = date, interval = interval)
            date = df.index[0]
        elif count % 200 != 0 :
            df = pyupbit.get_ohlcv(ticker, to = date, count = count % 200, interval = interval)
        else:
            break
        dfs.append(df)
        time.sleep(0.05)

    df = pd.concat(dfs).sort_index()

    return df

In [524]:
df = getdata(count = 1000)

In [525]:
df['gap'] = ((df['close'] / df['close'].shift(1)) - 1) * 100

df['ma5'] = df['close'].rolling(5).mean()
df = df.dropna()
raw_df = df.copy()
df = df.reset_index()

data = dict(df.T)

In [526]:
tick = 0
price1 = 0
tick_rate = 0 # 중간 틱의 인정범위
count = 0 # 틱의 시작

for idx, (key, elem) in enumerate(data.items()):
    data[idx]['tick'] = 0
    if idx == 0:
        data[idx]['tick'] = tick
        continue
        
    if tick in [1, 2, 3]:
        if data[idx]['gap'] > 0.1 or count == 4:
            tick = 0
            data[idx]['tick'] = tick
            tick_rate = 0
            count = 0
            continue
        
        else:
            if data[idx]['gap'] < -0.1 or tick_rate < - 0.15:
                tick += 1
                tick_rate = 0
                data[idx]['tick'] = tick
                print(str(tick) + "틱" + str(idx) + " " + "종가 : " + " " + str(data[idx]['close']))
                count = 0
                continue
            else:
                tick_rate += data[idx]['gap']
                count += 1
                continue
                
    else:
        if data[idx - 1]['close'] > data[idx - 1]['ma5'] and data[idx]['gap'] < -0.1 and tick == 0:
            tick = 1
            data[idx]['tick'] = tick
            
            print(str(tick) + "틱" + str(idx) + " " + "종가 : " + " " + str(data[idx]['close']))
            continue
        
    if tick == 4:
        print("시드 1/3 매수 진행")
        tick = 0
        tick_rate = 0
        count = 0
        data[idx]['tick'] = tick
        continue


dfnew = pd.DataFrame(data)
dfnew = dfnew.T
dfnew = dfnew.set_index('index')


1틱1 종가 :  40308000.0
1틱5 종가 :  40381000.0
1틱11 종가 :  40432000.0
2틱13 종가 :  40191000.0
3틱14 종가 :  39931000.0
4틱17 종가 :  39854000.0
시드 1/3 매수 진행
1틱21 종가 :  39908000.0
2틱23 종가 :  39800000.0
3틱24 종가 :  39556000.0
1틱28 종가 :  39518000.0
1틱30 종가 :  39600000.0
2틱31 종가 :  39497000.0
1틱37 종가 :  39661000.0
2틱38 종가 :  39539000.0
1틱46 종가 :  39722000.0
2틱47 종가 :  39482000.0
1틱50 종가 :  39630000.0
1틱61 종가 :  40002000.0
2틱62 종가 :  39933000.0
3틱63 종가 :  39890000.0
4틱65 종가 :  39758000.0
시드 1/3 매수 진행
1틱72 종가 :  39765000.0
2틱74 종가 :  39706000.0
1틱83 종가 :  39618000.0
1틱95 종가 :  39673000.0
2틱98 종가 :  39621000.0
1틱101 종가 :  39600000.0
1틱107 종가 :  39660000.0
2틱108 종가 :  39561000.0
3틱109 종가 :  39373000.0
1틱113 종가 :  39510000.0
1틱120 종가 :  39585000.0
1틱142 종가 :  39690000.0
1틱144 종가 :  39704000.0
1틱149 종가 :  39681000.0
1틱155 종가 :  39662000.0
1틱168 종가 :  39680000.0
1틱177 종가 :  39785000.0
2틱178 종가 :  39744000.0
1틱184 종가 :  39804000.0
2틱188 종가 :  39723000.0
1틱193 종가 :  39952000.0
2틱194 종가 :  39906000.0
3틱195 종가 :  3

In [527]:
len(data)

996

In [ ]:
# df = pyupbit.get_ohlcv()

kwargs = dict(title = "KRW-BTC" , type = 'candle', mav = (5,20,50), volume = True , ylabel= 'ohlc candles', figratio=(25,9))


#type='pnf'
mc = mpf.make_marketcolors(up = 'r' , down = 'b' , inherit = True)
mc1 = mpf.make_marketcolors(up = 'y' , down = 'k' , inherit = True)

s = mpf.make_mpf_style(marketcolors = mc)
s1 = mpf.make_mpf_style(marketcolors = mc)

scatter1 = mpf.make_addplot(np.where(dfnew['tick'] == 1 ,dfnew['close'] , np.nan) , type='scatter', marker='^', markersize=100, panel=0) #Issues drawing scatter
scatter2 = mpf.make_addplot(np.where(dfnew['tick'] == 2 ,dfnew['close'] , np.nan) , type='scatter', marker='v', markersize=100, panel=0) #Issues drawing scatter
scatter3 = mpf.make_addplot(np.where(dfnew['tick'] == 3 ,dfnew['close'] , np.nan) , type='scatter', marker='o', markersize=100, panel=0) #Issues drawing scatter
scatter4 = mpf.make_addplot(np.where(dfnew['tick'] == 4 ,dfnew['close'] , np.nan) , type='scatter', marker='x', markersize=100, panel=0) #Issues drawing scatter


plots = [scatter1, scatter2, scatter3, scatter4]
 
    
mpf.plot(raw_df, **kwargs, style = s, addplot=plots)

In [502]:
df


,index,open,high,low,close,volume,value,gap,ma5
0,2021-07-06 16:50:00,40258000.0,40309000.0,40255000.0,40308000.0,10.094206,4.065923e+08,0.131661,40283800.0
1,2021-07-06 16:55:00,40309000.0,40430000.0,40308000.0,40416000.0,52.968009,2.139727e+09,0.267937,40309000.0
2,2021-07-06 17:00:00,40424000.0,40425000.0,40320000.0,40365000.0,36.266392,1.464665e+09,-0.126188,40325800.0
3,2021-07-06 17:05:00,40369000.0,40370000.0,40330000.0,40361000.0,30.131005,1.215671e+09,-0.009910,40341000.0
4,2021-07-06 17:10:00,40363000.0,40398000.0,40305000.0,40308000.0,12.336338,4.979375e+08,-0.131315,40351600.0
5,2021-07-06 17:15:00,40307000.0,40350000.0,40269000.0,40338000.0,15.475655,6.236893e+08,0.074427,40357600.0
6,2021-07-06 17:20:00,40308000.0,40381000.0,40306000.0,40347000.0,13.761604,5.552722e+08,0.022311,40343800.0
7,2021-07-06 17:25:00,40348000.0,40450000.0,40329000.0,40428000.0,32.869647,1.328024e+09,0.200758,40356400.0
8,2021-07-06 17:30:00,40428000.0,40447000.0,40355000.0,40381000.0,22.020474,8.894608e+08,-0.116256,40360400.0
9,2021-07-06 17:35:00,40381000.0,40433000.0,40350000.0,40405000.0,21.709226,8.770883e+08,0.059434,40379800.0


In [296]:
# 알고리즘 구현

# data = 10000
# before_data = 9500
# idx = 0

# if gab < -0.2 and ma5 위이면서 다음 봉의 종가가 이전대비 하락으로 끝남 no counting: # 어짜피 상승장 조정이기에 쎄게 누르고 갈필요는 없을 듯?
#     idx = idx(gab)
#     check1 = first_price
#     tick = 1
    
#     if first_price와 next_price의 차이 > +0.1: #중간 양봉의 길이
#         pass
    
#     if first_price와 next_price의 차이 < 0.2  or 2 ~ 3 개의 갭의 곱이 -0.2 True:
#         tick = 2
#         second_price = next_price
        
#         if first_price와 next_price의 차이 > +0.1: #중간 양봉의 길이
#             pass
    
#         if second_price 와 next_price의 차이  < 0.2 or 2 ~ 3 개의 갭의 곱이 -0.2 True:
#             tick = 3
#             1/3 매수 진행 
#             third_price = next_price
            
#             20선 이탈 시 매도
#             if third_price <= -0.15:
#                 1/3매수 진행
                
#                 20선 이탈 시 매도
#                 if firth_price <= -0.15:
#                     1/3매수 진행
#                     20선 이탈시 매도 

    
# df['check3'] = np.where((df['gap'] < -0.15) & (df['check2'].shift(1)) ,df['close'], np.nan)

# df['gap'] < -0.15 and df['ma5'].shift(1) < df['close'].shift(1)

# if df['gap'] < -0.15 and df['ma5'].shift(1) < df['close'].shift(1):
#     df['ckeck1'] = 1
# else:
#     df['check1'] = 0




In [294]:
import yfinance as yf
import mplfinance as mpf
import numpy as np

yticker = yf.Ticker("MSFT")
data = yticker.history(period="1y") # max, 1y, 3mo, etc

def detect_intraday_price_diff(data):
    up_markers = []
    down_markers = []

    for index, row in data.iterrows():
        diff = row['Open'] - row['Close']
        diff_pct = diff / row['Open']

        if diff_pct >= 0.05:
          up_markers.append(row['Close'])
          down_markers.append(np.nan)
        elif diff_pct <= -0.05:
          down_markers.append(row['Close'])
          up_markers.append(np.nan)
        else:
          up_markers.append(np.nan)
          down_markers.append(np.nan)

    return up_markers, down_markers 
    

up_markers, down_markers = detect_intraday_price_diff(data)

up_plot = mpf.make_addplot(up_markers, type='scatter', marker='v', markersize=100, panel=0)
down_plot = mpf.make_addplot(down_markers, type='scatter', marker='^', markersize=100, panel=0)

plots = [up_plot, down_plot]

# mpf.plot(data, type='candle', style='yahoo', addplot=plots, title='MSFT')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)